In [40]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./src")
from scraper import Scraper
ganjoor = Scraper()
ganjoor.set_poets_name("https://ganjoor.net/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
with open("./data/ganjoor_poet_names.txt", "w") as f:
    for name in ganjoor.poet_names[:-1]:
        f.write(f"{name}\n")
    f.write(ganjoor.poet_names[-1])

In [42]:
poet_names = []
with open("./data/ganjoor_poet_names.txt","r") as f:
    for name in f.read().splitlines():
        poet_names.append(name)

In [43]:
poet_names.__len__()

191

In [44]:
def make_poets_url(names, cls):
    poets_url = [cls.extend_url(name) for name in names]
    return poets_url

In [45]:
poets_url = make_poets_url(ganjoor.poet_names, ganjoor)

In [46]:
import requests
from bs4 import BeautifulSoup
def get_first_layer(url):
    html_text = requests.get(url).text
    html_soup = BeautifulSoup(html_text,"lxml")

    divs_list = html_soup.find_all("div", class_="part-title-block")
    addresses = [str(div.find("a")) for div in divs_list]
    first_layer = [layer.split('"')[1].split("/")[-1] for layer in addresses]
    return first_layer

In [47]:
import requests
from bs4 import BeautifulSoup
def get_second_layer(url):
    html_text = requests.get(url).text
    html_soup = BeautifulSoup(html_text,"lxml")

    divs_list = html_soup.find_all("div", class_="part-title-block")
    addresses = [str(div.find("a")) for div in divs_list]
    second_layer = [layer.split('"')[1].split("/")[-1] for layer in addresses if "#index" not in layer]
    if second_layer:
        return second_layer
    else:
        return ["",]

In [48]:
poet_layers = dict()
poet_names_urls = list(zip(poet_names,poets_url))

for name, initial_layer in poet_names_urls:
    poet_layers[name] = list()
    
    for first_layer in get_first_layer(initial_layer):
        for second_layer in get_second_layer(f"{initial_layer}/{first_layer}"):
            if not second_layer:
                second_layer = ""
                slash = ""
            else:
                slash = "/"
            
            poet_layers[name].append(f"{initial_layer}/{first_layer}{slash}{second_layer}")
    print(name)
    


masood
roodaki
akhsikati
naziri
bahaee
saghir
emami
labibi
vatvat
ouhad
nezari
ayyooghi
asadi
hatef
fozooli
parvin
safi
khojandi
khaled
saadi
azraghi
shahedi
ghasem
saeb
babataher
kalim
seyf
hojviri
shahi
daye
elhami
mohit
moftagher
vahid
khayyam
yazdi
iraj
rhedayat
khalili
abolhasan
hajeb
homam
habib
maghrebi
feyz
hamid
safayee
ghobar
saber
fani
mohtasham
vaez
abusaeed
anvari
jalal
omman
rooni
fayyaz
forooghi
vahshi
ebnehesam
mmon
rafigh
asad
amagh
ghaani
neshat
razi
ouhadi
ghazzali
ghaemmagham
shooshtari
ghatran
valad
meybodi
babaafzal
kesayee
bahar
asrar
hoseyni
bahoo
iranshan
iqbal
amir
bolandeghbal
kamal
mahsati
hasan
aroozi
noorali
ghaleb
fasihi
vahdat
mahabadi
daghighi
onsori
ebneyamin
bidel
ahli
mir
eynolghozat
beylaghani
faghani
salman
shahrestani
heydar
toghra
osmani
mokhtari
tabib
khosro
jooya
azar
koosaj
falaki
jeyhoon
toghrol
shater
manoochehri
ghavami
ghari
khajenasir
hazin
safa
hamgar
masih
saeeda
eshghi
yaghma
koohi
kermani
moshtagh
naraghi
shahnematollah
rahi
zahiri
na

In [73]:
f = open("./data/second_layer_links", "w")
f.close()
poet_layers["rashhe"] = ["https://ganjoor.net/rashhe"]
for name, urls in poet_layers.items():
    if not urls:
        print(name)
    with open("./data/second_layer_links.txt", "a+") as f:
        for url in urls:
            f.write(f"{url}\n")

In [75]:
third_layer_links = []
with open("./data/second_layer_links.txt","r") as f:
    for link in f.read().splitlines():
        third_layer_links.append(link)

In [79]:
third_layer_links

['https://ganjoor.net/masood/divan-masood/ghasidemk',
 'https://ganjoor.net/masood/divan-masood/ghetemk',
 'https://ganjoor.net/masood/divan-masood/masnavimk',
 'https://ganjoor.net/masood/divan-masood/ghazalmk',
 'https://ganjoor.net/masood/divan-masood/tarkibjee',
 'https://ganjoor.net/masood/divan-masood/mosammatmk',
 'https://ganjoor.net/masood/divan-masood/mostazad',
 'https://ganjoor.net/masood/divan-masood/robaeemss',
 'https://ganjoor.net/masood/divan-masood/ghasidem',
 'https://ganjoor.net/masood/divan-masood/ghetem',
 'https://ganjoor.net/masood/towsifat/shahrashoob',
 'https://ganjoor.net/masood/towsifat/maah-farsi',
 'https://ganjoor.net/masood/towsifat/naam-e-roozha',
 'https://ganjoor.net/masood/towsifat/rooz-e-hafte',
 'https://ganjoor.net/roodaki/baghimande',
 'https://ganjoor.net/roodaki/robaeer',
 'https://ganjoor.net/roodaki/parakande',
 'https://ganjoor.net/roodaki/masnaviha/kalila-sand',
 'https://ganjoor.net/roodaki/masnaviha/motaghareb',
 'https://ganjoor.net/roo

In [93]:
html_text = requests.get("https://ganjoor.net/beylaghani/divan/ghaside").text
html_soup = BeautifulSoup(html_text,"lxml")
par_list = html_soup.find_all("p", class_="poem-excerpt")
addresses = [str(div.find("a")) for div in par_list]
links = [address.split('"')[1].split("/")[-1] for address in addresses]
links = list(set(links))
links

['sh64',
 'sh67',
 'sh66',
 'sh6',
 'sh44',
 'sh55',
 'sh49',
 'sh74',
 'sh20',
 'sh83',
 'sh3',
 'sh51',
 'sh77',
 'sh68',
 'sh58',
 'sh37',
 'sh1',
 'sh22',
 'sh78',
 'sh50',
 'sh43',
 'sh69',
 'sh32',
 'sh11',
 'sh82',
 'sh52',
 'sh25',
 'sh40',
 'sh5',
 'sh63',
 'sh27',
 'sh59',
 'sh24',
 'sh79',
 'sh9',
 'sh38',
 'sh45',
 'sh54',
 'sh16',
 'sh33',
 'sh39',
 'sh81',
 'sh71',
 'sh21',
 'sh57',
 'sh56',
 'sh18',
 'sh17',
 'sh73',
 'sh80',
 'sh35',
 'sh84',
 'sh15',
 'sh2',
 'sh65',
 'sh62',
 'sh14',
 'sh61',
 'sh23',
 'sh46',
 'sh60',
 'sh29',
 'sh53',
 'sh75',
 'sh76',
 'sh31',
 'sh13',
 'sh10',
 'sh8',
 'sh34',
 'sh19',
 'sh7',
 'sh28',
 'sh72',
 'sh26',
 'sh47',
 'sh42',
 'sh30',
 'sh70',
 'sh41',
 'sh12',
 'sh48',
 'sh36',
 'sh4']